# Working with multiple files

On occasion, we will need to combine more than 2 files using some combination of `UNION` and `JOIN`.  In this lecture, we will show a clean approach to scaling up these operations up to any number of files.  In the process, we will

1. Use `list` comprehensions to process and `UNION` many similar files.
2. Use `dict` comprehensions to store and access many tables by name.

In [15]:
import polars as pl

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## Example 1 - Read all baseball database using `dict`

**Task:** Create a `dict` of tables for all tables in the Lahman database

#### Step 1 - Use `glob` to find paths for all CSV files

In [16]:
from glob import glob
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core\\AllstarFull.csv',
 './data/baseball/core\\Appearances.csv',
 './data/baseball/core\\AwardsManagers.csv',
 './data/baseball/core\\AwardsPlayers.csv',
 './data/baseball/core\\AwardsShareManagers.csv',
 './data/baseball/core\\AwardsSharePlayers.csv',
 './data/baseball/core\\Batting.csv',
 './data/baseball/core\\BattingPost.csv',
 './data/baseball/core\\CollegePlaying.csv',
 './data/baseball/core\\Fielding.csv',
 './data/baseball/core\\FieldingOF.csv',
 './data/baseball/core\\FieldingOFsplit.csv',
 './data/baseball/core\\FieldingPost.csv',
 './data/baseball/core\\HallOfFame.csv',
 './data/baseball/core\\HomeGames.csv',
 './data/baseball/core\\Managers.csv',
 './data/baseball/core\\ManagersHalf.csv',
 './data/baseball/core\\Parks.csv',
 './data/baseball/core\\People.csv',
 './data/baseball/core\\Pitching.csv',
 './data/baseball/core\\PitchingPost.csv',
 './data/baseball/core\\Salaries.csv',
 './data/baseball/core\\Schools.csv',
 './data/baseball/core\\SeriesPost.csv'

#### Step 2 - Make a function to extract the table name

In [17]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) if FILE_NAME_RE.match(p) else None

[file_name(p) for p in files]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

#### 4 - Read in the tables.

In [18]:
baseball_db = {file_name(p):pl.read_csv(p, infer_schema_length=10000) for p in files}

baseball_db

{None: shape: (52, 10)
 ┌────────┬──────┬────────┬──────┬───┬──────┬─────┬─────┬─────┐
 │ yearID ┆ lgID ┆ teamID ┆ Half ┆ … ┆ Rank ┆ G   ┆ W   ┆ L   │
 │ ---    ┆ ---  ┆ ---    ┆ ---  ┆   ┆ ---  ┆ --- ┆ --- ┆ --- │
 │ i64    ┆ str  ┆ str    ┆ i64  ┆   ┆ i64  ┆ i64 ┆ i64 ┆ i64 │
 ╞════════╪══════╪════════╪══════╪═══╪══════╪═════╪═════╪═════╡
 │ 1981   ┆ NL   ┆ ATL    ┆ 1    ┆ … ┆ 4    ┆ 54  ┆ 25  ┆ 29  │
 │ 1981   ┆ NL   ┆ ATL    ┆ 2    ┆ … ┆ 5    ┆ 52  ┆ 25  ┆ 27  │
 │ 1981   ┆ AL   ┆ BAL    ┆ 1    ┆ … ┆ 2    ┆ 54  ┆ 31  ┆ 23  │
 │ 1981   ┆ AL   ┆ BAL    ┆ 2    ┆ … ┆ 4    ┆ 51  ┆ 28  ┆ 23  │
 │ 1981   ┆ AL   ┆ BOS    ┆ 1    ┆ … ┆ 5    ┆ 56  ┆ 30  ┆ 26  │
 │ …      ┆ …    ┆ …      ┆ …    ┆ … ┆ …    ┆ …   ┆ …   ┆ …   │
 │ 1981   ┆ NL   ┆ SLN    ┆ 2    ┆ … ┆ 2    ┆ 52  ┆ 29  ┆ 23  │
 │ 1981   ┆ AL   ┆ TEX    ┆ 1    ┆ … ┆ 2    ┆ 55  ┆ 33  ┆ 22  │
 │ 1981   ┆ AL   ┆ TEX    ┆ 2    ┆ … ┆ 3    ┆ 50  ┆ 24  ┆ 26  │
 │ 1981   ┆ AL   ┆ TOR    ┆ 1    ┆ … ┆ 7    ┆ 58  ┆ 16  ┆ 42  │
 │ 1981   ┆ AL   

### We can now access all the tables by name.

In [19]:
# Biggish output
baseball_db

{None: shape: (52, 10)
 ┌────────┬──────┬────────┬──────┬───┬──────┬─────┬─────┬─────┐
 │ yearID ┆ lgID ┆ teamID ┆ Half ┆ … ┆ Rank ┆ G   ┆ W   ┆ L   │
 │ ---    ┆ ---  ┆ ---    ┆ ---  ┆   ┆ ---  ┆ --- ┆ --- ┆ --- │
 │ i64    ┆ str  ┆ str    ┆ i64  ┆   ┆ i64  ┆ i64 ┆ i64 ┆ i64 │
 ╞════════╪══════╪════════╪══════╪═══╪══════╪═════╪═════╪═════╡
 │ 1981   ┆ NL   ┆ ATL    ┆ 1    ┆ … ┆ 4    ┆ 54  ┆ 25  ┆ 29  │
 │ 1981   ┆ NL   ┆ ATL    ┆ 2    ┆ … ┆ 5    ┆ 52  ┆ 25  ┆ 27  │
 │ 1981   ┆ AL   ┆ BAL    ┆ 1    ┆ … ┆ 2    ┆ 54  ┆ 31  ┆ 23  │
 │ 1981   ┆ AL   ┆ BAL    ┆ 2    ┆ … ┆ 4    ┆ 51  ┆ 28  ┆ 23  │
 │ 1981   ┆ AL   ┆ BOS    ┆ 1    ┆ … ┆ 5    ┆ 56  ┆ 30  ┆ 26  │
 │ …      ┆ …    ┆ …      ┆ …    ┆ … ┆ …    ┆ …   ┆ …   ┆ …   │
 │ 1981   ┆ NL   ┆ SLN    ┆ 2    ┆ … ┆ 2    ┆ 52  ┆ 29  ┆ 23  │
 │ 1981   ┆ AL   ┆ TEX    ┆ 1    ┆ … ┆ 2    ┆ 55  ┆ 33  ┆ 22  │
 │ 1981   ┆ AL   ┆ TEX    ┆ 2    ┆ … ┆ 3    ┆ 50  ┆ 24  ┆ 26  │
 │ 1981   ┆ AL   ┆ TOR    ┆ 1    ┆ … ┆ 7    ┆ 58  ┆ 16  ┆ 42  │
 │ 1981   ┆ AL   

In [20]:
baseball_db['Teams']

KeyError: 'Teams'

In [21]:
baseball_db['Batting']

KeyError: 'Batting'

## Example 2 - Reading and joining the baseball database using `dict`

**Task:** Collect the number of total hits for each batters in the 2010 season join on their first and last name.

In the second example, we will store the data frames in a `dict`, which will make it easier to join the files by ne

#### Step 1 - Get the files names

* Only need the `Batting.csv` and `People.csv`.  
* Narrow with a RegEx

In [22]:
from glob import glob
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core\\AllstarFull.csv',
 './data/baseball/core\\Appearances.csv',
 './data/baseball/core\\AwardsManagers.csv',
 './data/baseball/core\\AwardsPlayers.csv',
 './data/baseball/core\\AwardsShareManagers.csv',
 './data/baseball/core\\AwardsSharePlayers.csv',
 './data/baseball/core\\Batting.csv',
 './data/baseball/core\\BattingPost.csv',
 './data/baseball/core\\CollegePlaying.csv',
 './data/baseball/core\\Fielding.csv',
 './data/baseball/core\\FieldingOF.csv',
 './data/baseball/core\\FieldingOFsplit.csv',
 './data/baseball/core\\FieldingPost.csv',
 './data/baseball/core\\HallOfFame.csv',
 './data/baseball/core\\HomeGames.csv',
 './data/baseball/core\\Managers.csv',
 './data/baseball/core\\ManagersHalf.csv',
 './data/baseball/core\\Parks.csv',
 './data/baseball/core\\People.csv',
 './data/baseball/core\\Pitching.csv',
 './data/baseball/core\\PitchingPost.csv',
 './data/baseball/core\\Salaries.csv',
 './data/baseball/core\\Schools.csv',
 './data/baseball/core\\SeriesPost.csv'

#### Step 2 - Make helper functions to get the name from path

In [62]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/(Batting|People)\.csv$')
is_batting_or_people = lambda p: FILE_NAME_RE.match(p)
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 

[file_name(p) for p in files if is_batting_or_people(p)]

[]

In [46]:
dfs = [pl.read_csv(p) for p in files if is_batting_or_people(p)]

dfs

[]

#### Step 3 - Use a comprehension to read in all files

**Note:** The data is small (< 10mb total) so it is safe to read all at once.

In [48]:
dfs = {file_name(p):pl.read_csv(p) for p in files if is_batting_or_people(p)}
dfs['Batting'].head()

KeyError: 'Batting'

In [49]:
dfs['People'].head()

KeyError: 'People'

#### Step 4 - Preprocess each file.

In [50]:
# Filter, select, and aggregate hits for 2010.
hits_in_2010_raw = (dfs['Batting']
                   .select(['yearID', 'playerID', 'H'])
                   .filter(pl.col('yearID') == 2010)
                   .group_by('playerID')
                   .agg(pl.col('H').mean().alias('Total Hits'))
                   )
hits_in_2010_raw.head(2)

KeyError: 'Batting'

In [51]:
# Grab the first and last names from People.

player_names = (dfs['People']
                .select(['playerID', 'nameFirst', 'nameLast'])
               )
player_names.head(2)

KeyError: 'People'

#### Step 4 -- Join the tables

In [52]:
hits_in_2010 = (hits_in_2010_raw 
                .join(player_names, on='playerID', how='left')
                .drop('playerID')
               )
hits_in_2010.head()

NameError: name 'hits_in_2010_raw' is not defined

## <font color="red"> Exercise 3.2 </font>

We want to get the total hits allowed for all pitchers during the 2000-2010 seasons.  Use `glob` and a `dict` to collect this information into a table that includes the players first and last names.

In [53]:
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core\\AllstarFull.csv',
 './data/baseball/core\\Appearances.csv',
 './data/baseball/core\\AwardsManagers.csv',
 './data/baseball/core\\AwardsPlayers.csv',
 './data/baseball/core\\AwardsShareManagers.csv',
 './data/baseball/core\\AwardsSharePlayers.csv',
 './data/baseball/core\\Batting.csv',
 './data/baseball/core\\BattingPost.csv',
 './data/baseball/core\\CollegePlaying.csv',
 './data/baseball/core\\Fielding.csv',
 './data/baseball/core\\FieldingOF.csv',
 './data/baseball/core\\FieldingOFsplit.csv',
 './data/baseball/core\\FieldingPost.csv',
 './data/baseball/core\\HallOfFame.csv',
 './data/baseball/core\\HomeGames.csv',
 './data/baseball/core\\Managers.csv',
 './data/baseball/core\\ManagersHalf.csv',
 './data/baseball/core\\Parks.csv',
 './data/baseball/core\\People.csv',
 './data/baseball/core\\Pitching.csv',
 './data/baseball/core\\PitchingPost.csv',
 './data/baseball/core\\Salaries.csv',
 './data/baseball/core\\Schools.csv',
 './data/baseball/core\\SeriesPost.csv'

In [90]:
FILE_NAME_RE = re.compile(r'^(?:\.\/|\.\\)?data[\/\\]baseball[\/\\]core[\/\\](Pitching|People)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) if FILE_NAME_RE.match(p) else None 

In [91]:
baseball_db = {file_name(p): pl.read_csv(p, infer_schema_length=10000) for p in files if file_name(p) is not None}
baseball_db

{'People': shape: (20_090, 24)
 ┌───────────┬───────────┬────────────┬──────────┬───┬───────────┬───────────┬──────────┬───────────┐
 │ playerID  ┆ birthYear ┆ birthMonth ┆ birthDay ┆ … ┆ debut     ┆ finalGame ┆ retroID  ┆ bbrefID   │
 │ ---       ┆ ---       ┆ ---        ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---       │
 │ str       ┆ i64       ┆ i64        ┆ i64      ┆   ┆ str       ┆ str       ┆ str      ┆ str       │
 ╞═══════════╪═══════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
 │ aardsda01 ┆ 1981      ┆ 12         ┆ 27       ┆ … ┆ 2004-04-0 ┆ 2015-08-2 ┆ aardd001 ┆ aardsda01 │
 │           ┆           ┆            ┆          ┆   ┆ 6         ┆ 3         ┆          ┆           │
 │ aaronha01 ┆ 1934      ┆ 2          ┆ 5        ┆ … ┆ 1954-04-1 ┆ 1976-10-0 ┆ aaroh101 ┆ aaronha01 │
 │           ┆           ┆            ┆          ┆   ┆ 3         ┆ 3         ┆          ┆           │
 │ aaronto01 ┆ 1939      ┆ 8          ┆ 5        ┆ 

In [92]:
required_tables = ['Pitching', 'People']
dfs = {name: baseball_db[name] for name in required_tables if name in baseball_db}
required_tables
dfs

{'Pitching': shape: (47_628, 30)
 ┌───────────┬────────┬───────┬────────┬───┬─────┬──────┬──────┬──────┐
 │ playerID  ┆ yearID ┆ stint ┆ teamID ┆ … ┆ R   ┆ SH   ┆ SF   ┆ GIDP │
 │ ---       ┆ ---    ┆ ---   ┆ ---    ┆   ┆ --- ┆ ---  ┆ ---  ┆ ---  │
 │ str       ┆ i64    ┆ i64   ┆ str    ┆   ┆ i64 ┆ str  ┆ str  ┆ str  │
 ╞═══════════╪════════╪═══════╪════════╪═══╪═════╪══════╪══════╪══════╡
 │ bechtge01 ┆ 1871   ┆ 1     ┆ PH1    ┆ … ┆ 42  ┆ null ┆ null ┆ null │
 │ brainas01 ┆ 1871   ┆ 1     ┆ WS3    ┆ … ┆ 292 ┆ null ┆ null ┆ null │
 │ fergubo01 ┆ 1871   ┆ 1     ┆ NY2    ┆ … ┆ 9   ┆ null ┆ null ┆ null │
 │ fishech01 ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ 257 ┆ null ┆ null ┆ null │
 │ fleetfr01 ┆ 1871   ┆ 1     ┆ NY2    ┆ … ┆ 21  ┆ null ┆ null ┆ null │
 │ …         ┆ …      ┆ …     ┆ …      ┆ … ┆ …   ┆ …    ┆ …    ┆ …    │
 │ zamorda01 ┆ 2019   ┆ 1     ┆ NYN    ┆ … ┆ 5   ┆ 0    ┆ 1    ┆ 0    │
 │ zeuchtj01 ┆ 2019   ┆ 1     ┆ TOR    ┆ … ┆ 13  ┆ 0    ┆ 0    ┆ 1    │
 │ zimmejo02 ┆ 2019   ┆ 1     ┆

In [93]:
print(baseball_db.keys())

dict_keys(['People', 'Pitching'])


In [99]:
hits_in_2010_raw = (dfs['Pitching']
                    .select(['yearID', 'playerID', 'H'])
                    .filter((pl.col('yearID') >= 2000) & (pl.col('yearID') <= 2010))
                    .group_by('playerID')
                    .agg(pl.col('H').sum().alias('Total Hits')))

In [101]:
player_names = dfs['People'].select(['playerID', 'nameFirst', 'nameLast'])
total_hits_with_names = hits_in_2010_raw.join(player_names, on='playerID')
total_hits_with_names

playerID,Total Hits,nameFirst,nameLast
str,i64,str,str
"""aardsda01""",231,"""David""","""Aardsma"""
"""abadfe01""",14,"""Fernando""","""Abad"""
"""abbotpa01""",502,"""Paul""","""Abbott"""
"""abreuwi01""",57,"""Winston""","""Abreu"""
"""accarje01""",203,"""Jeremy""","""Accardo"""
…,…,…,…
"""zimmeje02""",128,"""Jeff""","""Zimmerman"""
"""zimmejo02""",126,"""Jordan""","""Zimmermann"""
"""zinkch01""",11,"""Charlie""","""Zink"""
